In [ ]:
from policyengine_us import Simulation

# Create hypothetical household
situation = {
  "people": {
    "you": {
      "age": {
        "2026": 40
      }
    },
    "your partner": {
      "age": {
        "2026": 40
      }
    },
    "your first dependent": {
      "age": {
        "2026": 10
      }
    },
    "your second dependent": {
      "age": {
        "2026": 10
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner",
        "your first dependent",
        "your second dependent"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent",
        "your second dependent"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner",
        "your first dependent",
        "your second dependent"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner",
        "your first dependent",
        "your second dependent"
      ],
      "state_name": {
        "2026": "CA"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    },
    "your first dependent's marital unit": {
      "members": [
        "your first dependent"
      ],
      "marital_unit_id": {
        "2026": 1
      }
    },
    "your second dependent's marital unit": {
      "members": [
        "your second dependent"
      ],
      "marital_unit_id": {
        "2026": 2
      }
    }
  },
  # Generate 200 sets of outputs, each with varying employment_income
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 200, # count of increments, not size
        "min": 0,
        "max": 200000
      }
    ]
  ]
}

simulation = Simulation(
    situation=situation,
)

# Get marginal tax rates of each member
output = simulation.calculate("marginal_tax_rate", 2026)

In [ ]:
# Get household size
size_array = simulation.calculate("household_size", 2026)

In [34]:
size_array[0]

4

In [ ]:
# Keep only head of household's output
mtrs_head = []
for i in range(output.shape[0]):
    if i % size_array[0] == 0:
        value = output[i]
        mtrs_head.append(value)

In [6]:
output.shape[0]

800

In [7]:
len(mtrs_head)

200

In [ ]:
# Define a regularization function
def regularize(x, threshold):
    if abs(x) < threshold:
        return 0
    else:
        return x

# Count the number of times the MTR graph changes direction
sign_switch_counter = 0
debug_list = []
for i in range(2, len(mtrs_head)):
    # first-difference of MTR, lagged once
        # If the first-difference is less than 1 percentage point, 
        # regularize() ignores the change.
    mtr_diff_l1 = regularize(mtrs_head[i-1] - mtrs_head[i-2], 1e-2)
    # first difference of MTR
    mtr_diff = regularize(mtrs_head[i] - mtrs_head[i-1], 1e-2)
    # Condition: if the two first differences swiched signs
    if mtr_diff_l1 * mtr_diff < 0:
        sign_switch_counter += 1
    debug_list.append(mtr_diff)

sign_switch_counter

5

In [ ]:
# Number of times sign-switches over the number of increments
sign_switch_counter/len(mtrs_head)

0.025